In [144]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [145]:
# read picture as GRAYSCALE
img=cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

In [146]:
# decide the corner type
def Yokoi_corner_type(b,c,d,e):
    if b==c and (d!=b or e!=b):
        return 'q'
    elif b==c and d==b and e==b:
        return 'r'
    elif b!=c:
        return 's'
    else:
        print("error! not a corner type!")

In [147]:
# decide the Yokoi number
def Yokoi_number(sub_image):
    dictionary = {'q': 0, 'r': 0, 's': 0}
    
    # top_right
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[1][2],sub_image[0][2],sub_image[0][1])]+=1
    # top_left
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[0][1],sub_image[0][0],sub_image[1][0])]+=1
    # buttom_left
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[1][0],sub_image[2][0],sub_image[2][1])]+=1
    # buttom_right
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[2][1],sub_image[2][2],sub_image[1][2])]+=1
    
    if dictionary['s'] == 4:
        return 0
    elif dictionary['q'] == 1:
        return 1
    elif dictionary['q'] == 2:
        return 2
    elif dictionary['q'] == 3:
        return 3
    elif dictionary['q'] == 4:
        return 4
    elif dictionary['r'] == 4:
        return 5
    else:
        print("error! not a yokoi number!")
        

In [148]:
# binary image (white->1 black->0)
def binarize(image):
    answer = np.zeros(image.shape, dtype=np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i][j]>127:
                answer[i][j] = 1
    return answer

In [149]:
# down sampleing(512*512 -> 64*64)
def down_sample(binary_image):
    answer = np.zeros((64,64), dtype=np.uint8)
    for i in range(64):
        for j in range(64):
            if binary_image[i*8][j*8]==1:
                answer[i][j] = 1
                
    return answer

In [150]:
# Interior_Border_Operator
# return: Border -> 1  ,Interior -> 0 
def Interior_Border_Operator(binary_image):
    position_list = [[0,1],[-1,0],[0,-1],[1,0]]
    # c->0  b->1 
    result = np.zeros(binary_image.shape, dtype = "int")
    
    # add outer frame to  the binary image
    binary_image_frame = np.zeros((binary_image.shape[0] + 2,binary_image.shape[1] + 2) , dtype = "int")
    binary_image_frame[1:binary_image_frame.shape[0]-1,1:binary_image_frame.shape[1]-1] = binary_image[0:binary_image.shape[0],0:binary_image.shape[1]]
     
    for i in range(1,binary_image_frame.shape[0]-1):
        for j in range(1,binary_image_frame.shape[1]-1):
            # 4-connected => 4 chance to detect
            if binary_image_frame[i][j] == 1:
                for k in range(4):
                    check_position_i = i + position_list[k][0]
                    check_opsition_j = j + position_list[k][1]
                
                    # border
                    if binary_image_frame[check_position_i][check_opsition_j]==0:
                        result[i-1][j-1] = 1
                        break
                    
    return result
                              

In [151]:
# pair_relationship_operator
# return: pixels which are border and has an interior neighbor will be labeled as 1 else 0
def pair_relationship_operator(I_B_image,binary_image):
    position_list = [[0,1],[-1,0],[0,-1],[1,0]]
    result = np.zeros(binary_image.shape,dtype = 'int')
    
    framed_I_B_image = np.ones((I_B_image.shape[0]+2,I_B_image.shape[1]+2),dtype='int')
    framed_I_B_image[1:framed_I_B_image.shape[0]-1,1:framed_I_B_image.shape[1]-1] = I_B_image[0:I_B_image.shape[0],0:I_B_image.shape[1]]
    framed_binary_image = np.zeros((binary_image.shape[0]+2,binary_image.shape[1]+2),dtype='int')
    framed_binary_image[1:framed_binary_image.shape[0]-1,1:framed_binary_image.shape[1]-1] = binary_image[0:binary_image.shape[0],0:binary_image.shape[1]]
    
    for i in range(1,framed_binary_image.shape[0]-1):
        for j in range(1,framed_binary_image.shape[1]-1):
            # if border => check it
            if framed_I_B_image[i][j]==1:
                for k in range(4):
                    check_point_i = i + position_list[k][0]
                    check_point_j = j + position_list[k][1]
                    
                    if framed_I_B_image[check_point_i][check_point_j]==0 and framed_binary_image[check_point_i][check_point_j]==1:
                        result[i-1][j-1]=1
    
    return result
    
    

In [152]:
# marked_pixel_connected_shrink_operator
# return thining result(white->1 black->0)
def marked_pixel_connected_shrink_operator(pair_image,binary_image):
    result_shape_0 = binary_image.shape[0]
    result_shape_1 = binary_image.shape[1]
    
    zero_padding_binary_image = np.zeros((result_shape_0 + 2,result_shape_1 + 2),dtype='int')
    zero_padding_binary_image[1:result_shape_0 + 1,1:result_shape_1 + 1] = binary_image[0:result_shape_0][0:result_shape_1]
    
    for i in range(1,result_shape_0 + 1):
        for j in range(1,result_shape_1 + 1):
            # it is white pixel
            if zero_padding_binary_image[i][j] == 1:
                yokoi_result_i_j =  Yokoi_number(zero_padding_binary_image[i-1:i+2,j-1:j+2])
                if pair_image[i-1][j-1] == 1 and yokoi_result_i_j == 1:
                    zero_padding_binary_image[i][j] = 0
    
    return  zero_padding_binary_image[1:result_shape_0+1,1:result_shape_1+1]
    
    

In [153]:
# input: binary image
# ouput: binary image after thining
def thining(binary_image):
    #result = np.zeros(binary_image.shape , dtype=np.uint8)
    #result[0:binary_image.shape[0],0:binary_image.shape[1]] = binary_image
    
    interior_border_image = Interior_Border_Operator(binary_image)
    paired_image = pair_relationship_operator(interior_border_image,binary_image)
    thining_image = marked_pixel_connected_shrink_operator(paired_image,binary_image)
    print(thining_image.shape)
                
    return thining_image
    
    

In [154]:
binary_image = binarize(img)
binary_sample_image = down_sample(binary_image)

result = thining(binary_sample_image)

for i in range(8):
    result = thining(result)





temp_image = np.zeros(result.shape,dtype = np.uint8)
for i in range(result.shape[0]):
    for j in range(result.shape[1]):
        if result[i][j]==1:
            temp_image[i][j]=255 
            
cv2.imwrite('answer1_5.bmp', temp_image)
temp_image = cv2.resize(temp_image, (512, 512))  
cv2.imshow('My Image',temp_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)


In [155]:
#down_sample_image = down_sample(binarize(img))
try_image = np.array([[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,0,0,0,0],[0,1,1,0,0]])





binary_image = binarize(img)
binary_image = down_sample(binary_image)

temp_image = np.zeros(binary_image.shape,dtype = np.uint8)
for i in range(binary_image.shape[0]):
    for j in range(binary_image.shape[1]):
        if binary_image[i][j]==1:
            temp_image[i][j]=255 

cv2.imwrite('answer1_1.bmp', temp_image)


I_B_image = Interior_Border_Operator(binary_image)
pair_image = pair_relationship_operator(I_B_image,binary_image)
thining_image = marked_pixel_connected_shrink_operator(pair_image,binary_image)

#print(I_B_image )
#print(pair_image)
#print(thining_image)

temp_image = np.zeros(binary_image.shape,dtype = np.uint8)
for i in range(binary_image.shape[0]):
    for j in range(binary_image.shape[1]):
        if I_B_image[i][j]==1:
            temp_image[i][j]=255 

final_image = np.zeros(binary_image.shape,dtype = np.uint8)
for i in range(binary_image.shape[0]):
    for j in range(binary_image.shape[1]):
        if thining_image[i][j]==1:
            final_image[i][j]=255 
            
cv2.imwrite('answer1_2.bmp', temp_image)

cv2.imwrite('answer1_3.bmp', final_image)

True